<a href="https://colab.research.google.com/github/brendadenisse16/Discrete-Optimization/blob/main/ML_Neural_Network_(allmodestransport).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#%pip install keras_tuner

In [3]:
from operator import mul
import itertools

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

import sklearn
import sklearn.ensemble    # For RandomForestClassifier class
import sklearn.metrics     # For accuracy_score

import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import StratifiedShuffleSplit  # to do stratified sampling
from sklearn.model_selection import train_test_split

from datetime import datetime
import math
from scipy.stats import skew
from scipy.stats import kurtosis

import seaborn; seaborn.set()  # set plot style
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import random
import pickle

from sklearn.preprocessing import LabelEncoder
import os

import importlib

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch

from sklearn.metrics import f1_score

In [5]:
df = pd.read_csv('20220615clean.csv')

In [6]:
total = len(df)

dataList = df.values.tolist()

Number_variable = 12 # number of variables 
Number_rawsessions = 6 # number of snapshots for aggregation
Gap_between_snapshot = 120000 #nano_seconds

In [7]:
Datanew = []

count = 0
for i in range(len(dataList)):
    #print(i)
    # Pour changer le nombre de snapshot à considérer
    if (i>=5 and i!=(len(dataList)-1)):
        if (
           #6 first snapshot have the same TM
            dataList[i][5]==dataList[i-1][5] and dataList[i][5]==dataList[i-2][5] 
            and dataList[i][5]==dataList[i-3][5] and dataList[i][5]==dataList[i-4][5] 
            and dataList[i][5]==dataList[i-5][5]  

            #TM are only Car
            #and dataList[i][5]!='walk' and dataList[i][5]!='run' and dataList[i][5]!='bike'
            # include all Transport Modes 

            #All the same user
            and dataList[i][7]==dataList[i-1][7] and dataList[i][7]==dataList[i-2][7] 
            and dataList[i][7]==dataList[i-3][7] and dataList[i][7]==dataList[i-4][7] 
            and dataList[i][7]==dataList[i-5][7]

            #Maximum gap  between snapshot...
            and (dataList[i][6]-dataList[i-1][6])<Gap_between_snapshot and (dataList[i-1][6]-dataList[i-2][6])<Gap_between_snapshot 
            and (dataList[i-2][6]-dataList[i-3][6])<Gap_between_snapshot and (dataList[i-3][6]-dataList[i-4][6])<Gap_between_snapshot 
            and (dataList[i-4][6]-dataList[i-5][6])<Gap_between_snapshot

            ):
            # [dataList[i][0]: PID
            # dataList[i][5]: Transport mode 
            X=[dataList[i][0], count, dataList[i][5]] #variables:PID, count y ActitivityType

            for j in range(Number_variable):
                if ((j+1) < 5): # con esto se que solo abarca :speed, heading, accuracy and altitde variables (col 1 a 4)
              #10 = distance -» Tout sauf distanceBus et Train apres mangent (6)
           #     if ((j+1) < 5 ):
                  data = [dataList[i-1][j+1],dataList[i][j+1],dataList[i-2][j+1],dataList[i-3][j+1],dataList[i-4][j+1],dataList[i-5][j+1]]  
                  x1 =  np.min(data)
                  x2 =  np.max(data)
                  x3 =  np.mean(data)
                  x4 =  np.std(data)
                  #new statistical features
                  x5 =  np.median(data)
                  x6 =  np.percentile((data),25) # percentile 25th
                  x7 =  np.percentile((data),75) # percentile 75th
                  x8 = x7 - x6 #interquartile range (perc75 - perc25)
                  x9 = x1 - x2 #value_range (max-min)

                  X = X+[x1,x2,x3,x4,x5,x6,x7,x8,x9]

                if ((j+1) >= 8 and (j+1) < 12): #con esto solo cubre las columnas: busDistanceStops, TrainDistance, BusCorridorDistance and MetroDistance
                  x1 = np.sum([dataList[i-1][j+1],dataList[i][j+1],dataList[i-2][j+1],dataList[i-3][j+1],dataList[i-4][j+1],dataList[i-5][j+1]])
                  X = X + [x1]
                if ((j+1) == 12 ): # se refiere a la variable "isAndroid"
                  x1 = dataList[i][j+1]
                  X = X + [x1]

            Datanew.append(X) 
            count = count + 1
            #print(X)
#random.shuffle(Datanew)

Features = pd.DataFrame(Datanew, columns = ['id', 'count', 'transportation_mode', 
                              'minSpeed' ,'maxSpeed', 'meanSpeed', 'stdSpeed','medianSpeed','25thPerSpeed','75thPerSpeed','inter_range_Speed','value_range_speed',
                              'minHeading' ,'maxHeading', 'meanHeading', 'stdHeading','medianHeading','25thPerHeading','75thPerHeading','inter_range_Heading','value_range_Heading',
                              'minAcc' ,'maxAcc', 'meanAcc', 'stdAcc', 'medianAcc','25thPerAcc','75thPerAcc','inter_range_Acc','value_range_Acc',
                              'minAlt' ,'maxAlt', 'meanAlt', 'stdAlt', 'medianAlt','25thPerAlt','75thPerAlt','inter_range_Alt','value_range_Alt',
                              'NumDist','NumDistTrain','NumDistBusCorridor' ,'NumDistMetro', 'IsAndroid'
                              ])


In [8]:
# Dimensions of new data frame with global statistical features
mrows=len(Features)
ncols=len(Features.columns)

data = Features.iloc[0:,2:ncols]
data['target_labeled'] = LabelEncoder().fit_transform(Features.iloc[0:,2])
data =data[['target_labeled','minSpeed' ,'maxSpeed', 'meanSpeed', 'stdSpeed','medianSpeed','25thPerSpeed','75thPerSpeed','inter_range_Speed','value_range_speed',
                              'minHeading' ,'maxHeading', 'meanHeading', 'stdHeading','medianHeading','25thPerHeading','75thPerHeading','inter_range_Heading','value_range_Heading',
                              'minAcc' ,'maxAcc', 'meanAcc', 'stdAcc', 'medianAcc','25thPerAcc','75thPerAcc','inter_range_Acc','value_range_Acc',
                              'minAlt' ,'maxAlt', 'meanAlt', 'stdAlt', 'medianAlt','25thPerAlt','75thPerAlt','inter_range_Alt','value_range_Alt',
                              'NumDist','NumDistTrain','NumDistBusCorridor','NumDistMetro', 'IsAndroid']]

In [9]:
Features

,id,count,transportation_mode,minSpeed,maxSpeed,meanSpeed,stdSpeed,medianSpeed,25thPerSpeed,75thPerSpeed,...,medianAlt,25thPerAlt,75thPerAlt,inter_range_Alt,value_range_Alt,NumDist,NumDistTrain,NumDistBusCorridor,NumDistMetro,IsAndroid
0,6,0,car,1.11,1.79,1.470000,0.279046,1.510,1.2100,1.7200,...,5.70,5.700,5.700,0.000,0.0,0,0,0,0,1
1,7,1,car,0.97,1.79,1.333333,0.289463,1.310,1.1100,1.5100,...,5.70,5.700,5.700,0.000,-1.3,0,0,0,0,1
2,8,2,car,0.97,1.51,1.196667,0.228813,1.110,1.0050,1.4100,...,5.70,5.700,6.675,0.975,-1.3,0,0,0,0,1
3,9,3,car,0.83,1.51,1.083333,0.213437,1.040,0.9700,1.1100,...,5.70,5.700,6.675,0.975,-3.0,0,0,0,0,1
4,10,4,car,0.83,1.11,0.970000,0.114310,0.970,0.8650,1.0750,...,5.70,4.425,6.675,2.250,-3.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231928,248788,231928,car,10.77,14.09,12.106667,1.250769,11.735,11.0475,13.0525,...,83.00,81.325,83.775,2.450,-4.2,0,0,0,0,0
231929,248789,231929,car,10.77,14.09,12.470000,1.148289,12.450,11.6725,13.3475,...,82.35,80.100,83.775,3.675,-4.5,0,0,0,0,0
231930,248790,231930,car,10.77,14.72,12.575000,1.309157,12.450,11.6725,13.3475,...,80.95,80.075,83.100,3.025,-4.5,0,0,0,0,0
231931,248791,231931,car,10.77,14.72,12.533333,1.284389,12.450,11.6725,13.1600,...,80.90,80.075,80.975,0.900,-4.5,0,0,0,0,0
